In [1]:
import psycopg2
import pandas as pd
import numpy as np
import json
import glob

In [5]:
liste_fichiers_2020 = glob.glob("import_data/2020*/*")
liste_fichiers_2021 = glob.glob("import_data/2021*/*")

len(liste_fichiers_2021+liste_fichiers_2020)

120405

In [7]:
len(glob.glob("import_data/*/*"))


120405

In [3]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vG&XGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "localhost", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 
 """
#  AND documentbudgetaire.nomenclature = 'M57-M57'
dat = pd.read_sql_query(query, conn)
conn.close()

/home/rbevenot/miniconda3/envs/dataprep/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
dat.describe()

,id,fk_id_document_budgetaire,id,exercice
count,4.236600e+04,42366.000000,42366.000000,42366.0
mean,6.560855e+05,114114.954138,114114.954138,2020.0
std,3.779747e+05,65808.434654,65808.434654,0.0
min,5.200000e+01,12.000000,12.000000,2020.0
25%,3.295655e+05,57304.250000,57304.250000,2020.0
50%,6.559225e+05,113929.000000,113929.000000,2020.0
75%,9.834660e+05,171020.000000,171020.000000,2020.0
max,1.308268e+06,227680.000000,227680.000000,2020.0


In [5]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [6]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]
#for index, row  in df.iterrows():
#    json_df = pd.read_json(row["json_annexe"])
#    json_df.assign(**row)



In [7]:
df_min

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""15000.00""}]",26180131000010,C.C.A.S du SUBDRAY,M14-M14_CCAS_INF3500,2020,Compte administratif,1,Principal,26180131000010,LE SUBDRAY - CCAS,Centre communal d'action sociale
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",20008077800018,SERPN LE THUIT ANGER,M4-M49_D,2020,Compte administratif,1,Principal,20008077800018,SERPN LE THUIT ANGER,Commune
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21830085300015,BUDGET COMMUNE,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21830085300015,COMMUNE DE LA MOTTE,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""LibCpte"": ""Pour ratio :...",21170244400038,BUDGET ANNEXE MULTISERVICE,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21170244400038,BUDGET ANNEXE MULTISERVICE,BUDGET ANNEXE MULTISERVICE
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",21790066100109,LOTISSEMENT CHAMPEAUX,M14-M14_COM_500_3500,2020,Compte administratif,1,Annexe,21790066100109,LOTISSEMENT CHAMPEAUX,OPÉRATIONS ET SERVICES ASSUJETTIS À LA TVA


In [10]:
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    #print(json_df)
    #json_df.assign(**row)

In [11]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('2020-CA.csv')  

In [16]:
df_to_csv['libelle_collectivite'].unique()

array(['ANJOU BLEU COMMUNAUTE', "Département de l'Eure",
       'CLERMONT AUVERGNE METROPOLE', 'CHARTRES METROPOLE',
       "NORMANDIE-CABOURG-PAYS D'AUGE", 'CC2SO', 'BOHAIN EN VERMANDOIS',
       'VILLE de BAVANS 25 550', 'SAINT-ETIENNE METROPOLE',
       'CC Val de Gray', "COM. de COM. DU VAL D'ARGENT",
       'CDC HAUTE SARTHE - ALPES MANCELLES', 'DEPARTEMENT DE L AISNE',
       'CA GRAND DOLE ', 'Toulouse Métropole',
       'Communaute Communes Plateau Picard', 'SUCE SUR ERDRE',
       'Toulon Provence Méditerranée', 'METZ METROPOLE',
       'Communauté Urbaine Caen La Mer', 'METROPOLE EUROPEENNE DE LILLE',
       'MAIRIE CERIZAY', 'Commune de St Georges les Bx',
       'Communauté de Communes du Pays de Fayence', 'VILLE DE TOURCOING',
       'VILLE DE STE MARIE-AUX-MINES', 'LOTISSEMENT LES EPINATS',
       'ORLEANS METROPOLE', 'GRENOBLE ALPES METROPOLE',
       "COM. D'AGGLO. VALENCIENNES METROPOLE",
       'MONTPELLIER MEDITERRANEE METROPOLE', 'C.C. DOUBS BAUMOIS',
       'Commun

In [9]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']

In [10]:
parsed_df = pd.concat([pd.read_json(js, orient='records') for js in df_min['json_annexe']])

AttributeError: 'NoneType' object has no attribute 'keys'

In [7]:
with open("./M71_2020/99_BU-076-200053403-20191216-83959A-BF-1-1_1.xml", encoding='latin-1') as fd:
    df = pd.read_xml(fd.read(), namespaces={"doc": "http://www.minefi.gouv.fr/cp/demat/docbudgetaire"})

In [8]:
df

,V,CodeEditeur,outil,version,md5,sha1,date,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,EnTeteBudget,BlocBudget,InformationsGenerales,LigneBudget,Annexes
0,86.0,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,DSC_COMMON,27,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,REMAT,6_2019,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,None,None,242DFB401EB3F04A8FDD3ED067A27876,66E377B3F5089093F4838F3591EB6A8A7F719B7A,2019-12-19T14:51:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
from functools import reduce

def json_to_df(row):
    _, row = row
    df_json = pd.read_json(row["json_annexe"])
    col1 = pd.Series([row.exercice]*len(df_json), name='exercice')
    return pd.concat([col1,df_json],axis=1)

df = map(json_to_df, df.iterrows())      
df = reduce(lambda x,y:x.append(y), x)

NameError: name 'x' is not defined

In [54]:
json_df.assign(row.drop("json_annexe"))

TypeError: assign() takes 1 positional argument but 2 were given

In [8]:
df = pd.read_xml("./M71_2020/99_BU-021-200053726-20191211-BP_2020-BF-1-1_1.xml",
                 encoding='iso-8859-1',
                 #parser='etree',
                 #attrs_only=True,
                 xpath="/xmlns:DocumentBudgetaire/xmlns:Budget/xmlns:Annexes/xmlns:DATA_APCP//xmlns:APCP",
                namespaces={"xmlns": "http://www.minefi.gouv.fr/cp/demat/docbudgetaire"}
                )

nan

In [7]:
dfs = []
def json_to_df(row, json_col):
    json_df = pd.read_json(row[json_col])
    dfs.append(json_df.assign(**row.drop(json_col)))
 
df.apply(json_to_df, axis=1, json_col='json_annexe')
df_glob = pd.concat(dfs)

AttributeError: 'NoneType' object has no attribute 'keys'

In [29]:
dfs

[     CodTypEmpr CodProfilAmort CodProfilAmortDtVote  CodArticle  AnEncaisse  \
 0             1              X                    X     16441.0      2007.0   
 1             1              X                    X     16441.0      2005.0   
 2             1              C                    C      1641.0      2017.0   
 3             1              C                    C      1641.0      2018.0   
 4             1              C                    C      1641.0      2014.0   
 ..          ...            ...                  ...         ...         ...   
 104           9              P                  NaN         NaN      2006.0   
 105           9              P                  NaN         NaN      2006.0   
 106           9              P                  NaN         NaN      2014.0   
 107           9              P                  NaN         NaN      1995.0   
 108           9              P                  NaN         NaN      1990.0   
 
       MtEmprOrig  DureeRest          